In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

from shapely.geometry import Point, Polygon, LineString

from tqdm.auto import tqdm

In [ ]:
trees = gpd.read_file('../data/yerevan_trees.geojson')

In [ ]:
buildings = gpd.read_file('../data/yerevan_buildings_with_id.geojson')
buildings_clean = gpd.read_file('../data/yerevan_buildings_clean.geojson')

In [4]:
buildings = buildings.merge(buildings_clean[['osmid', 'img_id']], on=['osmid', 'img_id'], how='inner')

In [ ]:
districts = gpd.read_file('../data/yerevan_districts.geojson')

In [6]:
## FILTER FOR DISTRICT

trees = trees.sjoin(districts[districts['name'] == 'Kentron'][['geometry']], predicate='intersects').reset_index(drop=True)
trees.drop(columns=['index_right'], inplace=True)

buildings = buildings.sjoin(districts[districts['name'] == 'Kentron'][['geometry']], predicate='within')
buildings.drop(columns=['index_right'], inplace=True)

buildings = buildings[['building_id', 'img_id', 'name', 'geometry']].reset_index(drop=True)

In [7]:
trees

,img_id,geometry
0,1101,POINT (44.49594 40.15967)
1,1101,POINT (44.49619 40.15971)
2,1101,POINT (44.49618 40.15948)
3,1101,POINT (44.49677 40.15973)
4,1101,POINT (44.49792 40.15937)
...,...,...
25895,657,POINT (44.54078 40.19547)
25896,657,POINT (44.53938 40.19547)
25897,657,POINT (44.54442 40.19546)
25898,657,POINT (44.54108 40.19547)


In [8]:
def count_trees(trees_gdf, buildings_gdf):
    """
    Tags each tree with building_id(s) it is visible from based on circular visibility
    from the centroid of the building with a radius equal to the building's size (length/width).

    Parameters:
    - trees_gdf: GeoDataFrame of tree Points (EPSG:4326)
    - buildings_gdf: GeoDataFrame of building Polygons with a 'building_id' column (EPSG:4326)

    Returns:
    - buildings_with_counts: GeoDataFrame of buildings with 'visible_trees' count
    - visible_trees_with_building_ids: GeoDataFrame of all trees with 'building_id' (can be None or multiple)
    """
    
    # Project to metric CRS (for accurate distance measurements)
    buildings_proj = buildings_gdf.to_crs(epsg=3857).copy()
    trees_proj = trees_gdf.to_crs(epsg=3857).copy()

    buildings_proj["visible_trees"] = 0
    tree_visibility = {i: [] for i in trees_proj.index}  # collect all visible building_ids per tree

    for _, building in tqdm(buildings_proj.iterrows(), total=len(buildings_proj)):
        b_id = building["building_id"]
        building_geom = building.geometry

        # Calculate centroid of the building
        centroid = building_geom.centroid

        # Calculate the building's length and width (bbox)
        minx, miny, maxx, maxy = building_geom.bounds
        length = maxx - minx
        width = maxy - miny
        
        # Set the buffer radius to the maximum of the length and width of the building
        buffer_radius = max(length, width)

        # Create a circular buffer around the centroid
        buffer_zone = centroid.buffer(buffer_radius)

        # Find trees within the buffer zone
        nearby_trees = trees_proj[trees_proj.geometry.within(buffer_zone)]

        # Collect visible trees' indices for this building
        for _, tree in nearby_trees.iterrows():
            tree_visibility[tree.name].append(b_id)

        # Count the number of visible trees for this building
        visible_tree_indices = set()
        for _, tree in nearby_trees.iterrows():
            visible_tree_indices.add(tree.name)
        
        buildings_proj.loc[buildings_proj["building_id"] == b_id, "visible_trees"] = len(visible_tree_indices)

    # Build the output GeoDataFrame for trees with visible building IDs
    tree_records = []
    for i, row in trees_proj.iterrows():
        building_ids = tree_visibility[i]
        if not building_ids:
            tree_records.append({"building_id": None, "geometry": row.geometry})
        else:
            for b_id in building_ids:
                tree_records.append({"building_id": b_id, "geometry": row.geometry})

    visible_trees_gdf = gpd.GeoDataFrame(tree_records, geometry="geometry", crs=trees_proj.crs)

    # Return to original CRS
    return buildings_proj.to_crs(epsg=4326), visible_trees_gdf.to_crs(epsg=4326)

In [ ]:
buildings_with_counts, visible_trees_with_building_ids = count_trees(trees, buildings)

In [10]:
visible_trees_with_building_ids

,building_id,geometry
0,NaN,POINT (44.49594 40.15967)
1,NaN,POINT (44.49619 40.15971)
2,NaN,POINT (44.49618 40.15948)
3,NaN,POINT (44.49677 40.15973)
4,NaN,POINT (44.49792 40.15937)
...,...,...
43393,NaN,POINT (44.54078 40.19547)
43394,72390.0,POINT (44.53938 40.19547)
43395,NaN,POINT (44.54442 40.19546)
43396,NaN,POINT (44.54108 40.19547)


In [11]:
buildings_with_counts

,building_id,img_id,name,geometry,visible_trees
0,9982,1120,None,"POLYGON ((44.49033 40.15967, 44.49042 40.15968...",1
1,10223,1122,Հանրապետական մանկական վերականգնողական կենտրոն,"POLYGON ((44.49042 40.16255, 44.49049 40.16224...",16
2,10224,1122,None,"POLYGON ((44.49003 40.16252, 44.49023 40.16252...",0
3,10225,1122,None,"POLYGON ((44.48987 40.16214, 44.48999 40.16217...",5
4,10226,1122,None,"POLYGON ((44.49052 40.16183, 44.49072 40.16183...",3
...,...,...,...,...,...
9058,72401,657,None,"POLYGON ((44.54391 40.19578, 44.54390 40.19569...",0
9059,72402,657,None,"POLYGON ((44.54384 40.19601, 44.54389 40.19599...",3
9060,72403,657,None,"POLYGON ((44.54449 40.19583, 44.54454 40.19579...",4
9061,72404,657,None,"POLYGON ((44.54468 40.19582, 44.54463 40.19579...",5


In [ ]:
visible_trees_with_building_ids.to_file('../trees_with_buildins_ids_kentron.geojson')

In [ ]:
buildings_with_counts.to_file('../buildings_with_tree_counts_kentron.geojson')